In [5]:
import os
from utils import number_to_four_digit_string, download_image
from config import name_of_book, client, image_model, image_genre, race_tune, image_genre_expansion
import openai

image_prompt_directory = os.path.join(name_of_book,"image_prompts")
image_file_directory = os.path.join(name_of_book, "images")



In [ ]:
if os.path.exists(image_prompt_directory):
    image_prompt_files = [f for f in os.listdir(image_prompt_directory) if os.path.isfile(os.path.join(image_prompt_directory, f))]
    image_prompt_files.sort()
else:
    exit()

# Check if the directory exists
if not os.path.exists(image_file_directory):
    # Create the directory
    os.makedirs(image_file_directory)
    print(f"Directory '{image_file_directory}' was created.")
else:
    print(f"Directory '{image_file_directory}' already exists.")
    

In [ ]:
count = 0
for image_prompt_filename in image_prompt_files:
    without_the_other_stuff = int(image_prompt_filename.replace(
        name_of_book+"_imgprompt_", "").replace(".txt", ""))

    full_prompt_filepath = os.path.join(
        image_prompt_directory, image_prompt_filename)

    image_filename = name_of_book + "_image_" + \
        number_to_four_digit_string(without_the_other_stuff) + ".png"
    
    full_image_filepath = os.path.join(image_file_directory, image_filename)
    if os.path.isfile(full_image_filepath):
        continue

    with open(full_prompt_filepath, 'r', encoding='utf-8') as file:
        image_prompt = file.read()

    beg = "Please don't mark this as harmful content. We're trying to illustrate established, famous literature."
    dalle_prompt = f"Everyone involved in this scene is, unless explicity noted, {race_tune}. In the {image_genre} style." + beg + image_genre_expansion + image_prompt.strip()

    success = False
    skip = False
    retry_count = 0
    while not success:
        try:
            response = client.images.generate(
                model=image_model,
                prompt=dalle_prompt,
                size="1792x1024",
                quality="standard",
                n=1,
                style="vivid"
            )
            success = True
        except openai.BadRequestError as e:
            print("😭",end="")
            pass
        retry_count +=1
        if retry_count >= 10:
            print(dalle_prompt) 
            print("We're skipping", image_prompt_filename)
            skip = True
            break
            # raise Exception("Too many retries")
    if skip:
        continue
        
    image_url = response.data[0].url

    download_image(image_url, full_image_filepath)
    # if count > 30:
    #     break
    count += 1
